In [4]:
from pathlib import Path

import pandas as pd
from codetf.models import load_model_pipeline
from sacrebleu.metrics import BLEU
from datasets import load_dataset

In [5]:
root_dir = Path.cwd()

In [6]:
dataset = load_dataset("json", data_files={
    "test": str(root_dir / "data" / "test.jsonl"),
}, cache_dir=root_dir / "data" / "cache")

Found cached dataset json (/home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-acdd91729f392843/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 17.94it/s]


In [7]:
def inference(doc):
    doc["ref"] = [" ".join(docstring) for docstring in doc["docstring_tokens"]]
    return doc    

dataset = dataset.map(inference, batched=True)

In [9]:
dataset.set_format(type="pandas", columns=["ref", "code"])
df = dataset["test"][:]
df.head()

,code,ref
0,def sina_xml_to_url_list(xml_data):\n rawur...,str - > list Convert XML to URL List . From Bi...
1,"def dailymotion_download(url, output_dir='.', ...",Downloads Dailymotion videos by URL .
2,"def sina_download(url, output_dir='.', merge=T...",Downloads Sina videos by URL .
3,"def sprint(text, *colors):\n return ""\33[{}...",Format text with color or other effects into A...
4,"def print_log(text, *colors):\n sys.stderr....",Print a log message to standard error .


In [29]:
model = load_model_pipeline(model_name="codet5", task="sum_python", model_type="base")
preds = []

for i, row in df.head(100).iterrows():
    if i % 100 == 0: print(i)
    preds.extend(model.predict([row["code"]]))

0


Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors


In [25]:
refs = df["ref"].head(100).tolist()

In [33]:
bleu = BLEU()
bleu.corpus_score(preds, [refs]).__dict__

{'name': 'BLEU',
 'score': 9.090378120536718,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '47.3/18.4/9.6/5.3 (BP = 0.626 ratio = 0.681 hyp_len = 770 ref_len = 1131)',
 'bp': 0.6257332175093627,
 'counts': [364, 123, 55, 25],
 'totals': [770, 670, 570, 470],
 'sys_len': 770,
 'ref_len': 1131,
 'precisions': [47.27272727272727,
  18.35820895522388,
  9.649122807017545,
  5.319148936170213],
 'prec_str': '47.3/18.4/9.6/5.3',
 'ratio': 0.6808134394341291}